# Hierarchical Clustering

## 1. Naive Implementation

This implementation performs hierarchical clustering using a naive greedy approach. Initially each data point is considered as an individual cluster. At each iteration, the similar clusters merge with other clusters until 1/ K clusters are formed.

There is no need to specify the number of clusters, which is an advantage, however this comes with a price: performance $O(n^{3}).$

In [1]:
import numpy as np
import pandas as pd

In [2]:
# the distance function defined as the euclidian distance between two points
def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2) # euclidean distance is the l2 norm, with the default value of the 'ord' as 2

# initialize the distance matrix between all points
def initialize_distance_matrix(points):
    from scipy.spatial.distance import pdist, squareform
    return squareform(pdist(points))

def find_closest_clusters(clusters, distance_matrix):
    min_distance = np.inf
    closest_pair = None
    # find two nearest clusters c1 and c2 in C, with single linkage
    for i in range(len(clusters)):
        for j in range(i + 1, len(clusters)):
            dist = min(distance_matrix[a][b] for a in clusters[i] for b in clusters[j])
            if dist < min_distance:
                min_distance = dist
                closest_pair = (i, j)
    return closest_pair, min_distance

def naive_hc(points):
    num_points = len(points)
    clusters = [{i} for i in range(num_points)]
    distance_matrix = initialize_distance_matrix(points)
    merge_order = []
    
    # merge the two closest clusters
    while len(clusters) > 1:
        (i, j), min_distance = find_closest_clusters(clusters, distance_matrix)
        merge_order.append((list(clusters[i]), list(clusters[j]), min_distance))
        clusters.append(clusters[i] | clusters[j])
        # remove the merged clusters
        clusters.pop(max(i, j))
        clusters.pop(min(i, j))

    return clusters[0], merge_order

## 2. Applying to Dataset

### Load and Preprocess Data

In [3]:
raw_df = pd.read_csv('CC_GENERAL.csv') # load dataset
raw_df = raw_df.drop('CUST_ID', axis = 1) # removing the 'Customer ID' label column
raw_df.ffill(inplace=True) # replace the missing values with the last valid (non-null) value
raw_df.head()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,627.284787,0.000000,12
4,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [4]:
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [5]:
# standardize data
scaler = StandardScaler() 
scaled_df = scaler.fit_transform(raw_df) 
  
# normalizing the data 
normalized_df = normalize(scaled_df) 
  
# converting the numpy array into a pandas DataFrame 
normalized_df = pd.DataFrame(normalized_df) 

# since we are using a large dataset, we will reduce the dimensions of the data 
pca = PCA(n_components = 2) # reduce to 2 dimensions
X_principal = pca.fit_transform(normalized_df) 
X_principal = pd.DataFrame(X_principal) 
X_principal.columns = ['P1', 'P2'] 
  
X_principal.head(2)

,P1,P2
0,-0.489949,-0.679976
1,-0.519099,0.544826


We have:

- Standardized the data using StandardScaler to ensure each feature contributes equally by having a mean of 0 and a standard deviation of 1.
- Normalized the data to ensure the data attributes have the same scale. This can help improve the performance of many machine learning algorithms.
- Performed dimensionality reduction using PCA, reducing the dataset to 2 principal components for simplicity and easier visualization.

#### Applying our hierarchical clustering function to randomly generated data:
In the following, we first demonstrate our function with a set of randomly generated data points. 

In [6]:
# generate random data points
np.random.seed(42)  # for reproducibility
random_points = np.random.rand(10, 2) * 100  # 10 points in 2D space, scaled by 100

# use the hierarchical clustering function
clusters, merge_order = naive_hc(random_points)

# print the results
for step, (cluster1, cluster2, dist) in enumerate(merge_order, 1):
    print(f"Merge step {step}: Cluster {cluster1} and {cluster2} (distance: {dist:.2f})")


Merge step 1: Cluster [2] and [7] (distance: 3.76)
Merge step 2: Cluster [3] and [5] (distance: 11.03)
Merge step 3: Cluster [1] and [4] (distance: 17.06)
Merge step 4: Cluster [8] and [9] (distance: 26.62)
Merge step 5: Cluster [2, 7] and [8, 9] (distance: 27.24)
Merge step 6: Cluster [0] and [3, 5] (distance: 32.76)
Merge step 7: Cluster [1, 4] and [0, 3, 5] (distance: 33.20)
Merge step 8: Cluster [8, 9, 2, 7] and [0, 1, 3, 4, 5] (distance: 34.89)
Merge step 9: Cluster [6] and [0, 1, 2, 3, 4, 5, 7, 8, 9] (distance: 39.92)


In [7]:
# Because the naive hierarchical clustering is computationally intensive,
# using a small subset of points for demonstration: 
data_sample = X_principal.sample(n=500, random_state=1).values
# randomly selects n rows from the DataFrame X_principal, with a set random state, and converted to numpy array

# applying the hierarchical clustering function
clusters, merge_order = naive_hc(data_sample)
    
# print some of the merge orders
for step, (cluster1, cluster2, dist) in enumerate(merge_order[-10:], 1):  # just the last 10 merges
    print(f"Merge step {step}: Cluster {cluster1} and {cluster2} (distance: {dist:.2f})")

Merge step 1: Cluster [407] and [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209, 210, 211, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 22

In [8]:
# # scatter plot of the two principal components
# plt.scatter(X_principal['P1'], X_principal['P2'], alpha=0.6)
# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.title('Data After PCA')
# plt.show()

## 3. Built-in Implementation

### Using sklearn.cluster.AgglomerativeClustering for Hierarchical Clustering

In this section, we will be implementing hierarchical clustering by using built-in implementation of hierarchical clustering to compare our results. Agglomerative Clustering, which is available in scikit-learn Python library, recursively merges pair of clusters of sample data; uses linkage distance. In sklearn’s implementation, we can specify the number of clusters to assist the algorithm’s performance.

The AgglomerativeClustering object performs a hierarchical clustering using a bottom up approach: each observation starts in its own cluster, and clusters are successively merged together. The linkage criteria determines the metric used for the merge strategy:

- Ward minimizes the sum of squared differences within all clusters. It is a variance-minimizing approach and in this sense is similar to the k-means objective function but tackled with an agglomerative hierarchical approach.

- Maximum or complete linkage minimizes the maximum distance between observations of pairs of clusters.

- Average linkage minimizes the average of the distances between all observations of pairs of clusters.

- Single linkage minimizes the distance between the closest observations of pairs of clusters.

AgglomerativeClustering can also scale to large number of samples when it is used jointly with a connectivity matrix, but is computationally expensive when no connectivity constraints are added between samples: it considers at each step all the possible merges.

In [9]:
# import time
#from sklearn.cluster import AgglomerativeClustering 
#from scipy.cluster.hierarchy import dendrogram, linkage
#from sklearn.metrics import silhouette_score

In [10]:
# plt.figure(figsize =(6, 6)) 
# plt.title('Visualising the data') 
# Dendrogram = dendrogram((linkage(X_principal, method ='ward'))) 

In [11]:
# Determine the optimal number of clusters using [Silhouette Score]

In [12]:
# silhouette_scores = [] 

# for n_cluster in range(2, 8):
#     silhouette_scores.append( 
#         silhouette_score(X_principal, AgglomerativeClustering(n_clusters = n_cluster).fit_predict(X_principal))) 
    
# # Plotting a bar graph to compare the results 
# k = [2, 3, 4, 5, 6,7] 
# plt.bar(k, silhouette_scores) 
# plt.xlabel('Number of clusters', fontsize = 10) 
# plt.ylabel('Silhouette Score', fontsize = 10) 
# plt.show() 

In [13]:
# agg = AgglomerativeClustering(n_clusters=3)
# agg.fit(X_principal)

In [14]:
# # Visualizing the clustering 
# plt.scatter(X_principal['P1'], X_principal['P2'],  
#            c = AgglomerativeClustering(n_clusters = 3).fit_predict(X_principal), cmap =plt.cm.winter) 
# plt.show() 

### Comparing Execution Time

In [15]:
# def time_function(func, *args):
#     start_time = time.time()
#     func(*args)
#     end_time = time.time()
#     return end_time - start_time

# # Timing your custom hierarchical clustering
# custom_clustering_time = time_function(naive_hc, data_sample)

# # Timing AgglomerativeClustering
# sklearn_clustering_time = time_function(AgglomerativeClustering, data_sample)

# print(f"Custom Hierarchical Clustering took: {custom_clustering_time:.2f} seconds")
# print(f"Scikit-Learn Agglomerative Clustering took: {sklearn_clustering_time:.2f} seconds")


### Resources

- https://www.kaggle.com/code/vipulgandhi/hierarchical-clustering-explanation/notebook
- https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering
- https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html